In [ ]:
import datetime as dt
import pandas as pd

In [9]:
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas52semanas.parquet")

In [10]:
df.sample(10)

,price_subtotal,company_id,id,quantity,product_id_num,product_name,invoice_id_num,invoice_name,date_invoice
116996,27000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1472592,1.0,47435,[DCE00764189] G764 FILTRO SEPARDOR COMBUSTIBLE...,400511,FCOT23041,2025-02-17 13:23:16
48540,212400.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1553746,6.0,49905,[DCS10197189] GS197A1 FILTRO COMBUSTIBLE SEPAR...,421937,PV1E10342 Most Cota13856,2025-06-18 12:22:14
186730,42800.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1389809,1.0,16994,[DAR02499025] DA2499 FILTRO AIRE DONSSON - Che...,378631,PV2E33753 Mostrador Calle 6 T1/21440,2024-10-04 14:33:00
35994,152600.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1569902,2.0,20813,[BCE00782125] G782 FILTRO COMBUSTIBLE HITACHI ...,425966,FBUC6271,2025-07-11 16:46:43
48556,258000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1553731,2.0,18908,[BLS00286125] GS286 FILTRO ACEITE DUAL MOTORES...,421935,FCOT24919,2025-06-17 22:00:57
73909,82600.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1523890,2.0,17308,[DAE02983025] DA2983 FILTROAIRE MOTORES PERKIN...,414050,FVAL691,2025-05-05 16:59:16
182949,64700.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1394046,1.0,20749,[BCE00663125] G663 FILTRO COMBUSTIBLE BALDWIN ...,379864,PV3E15501 Mostrador Valladolid T1/20652,2024-10-11 12:37:50
86588,31500.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1509650,1.0,17058,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,409988,FCOT23894,2025-04-09 13:59:04
27091,184900.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1580146,1.0,49890,[DAR08242025] DA8242 FILTRO AIRE - FOTON (02...,428776,FMDE3412,2025-07-26 13:13:07
184401,279000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1392364,5.0,20757,[BCE00764125] G764 FILTRO COMB 25 MICRONES DA...,379405,FCOT21155,2024-10-09 13:37:39


In [11]:
import re
import unicodedata

# Diccionario de códigos a sucursales
mapa_codigos = {
    "FCAL": "CALI",
    "FMDE": "MEDELLIN",
    "FBOG": "BOGOTA",
    "FCTG": "CARTAGENA",
    "FBAQ": "BARRANQUILLA",
    "FVAL":"VALLADOLID",
    "FCOT":"VENDEDOR EXTERNO",
    "FBUC":"BUCARAMANGA",
    "FNOR":"NORTE"
}


# Equivalencias para normalizar nombres truncados o mal escritos
mapa_equivalencias = {
    "MEDELLIN": "MEDELLIN",
    "MEDELLI": "MEDELLIN",
    "MEDELL": "MEDELLIN",
    "MEDELI": "MEDELLIN",
    "CALI": "CALI",
    "BARRANQUILLA": "BARRANQUILLA",
    "BOGOTA": "BOGOTA",
    "CARTAGENA": "CARTAGENA",
    "VALLADOLID": "VALLADOLID"
}

def normalizar(texto):
    """Quita tildes y pasa a mayúsculas"""
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join([c for c in texto if not unicodedata.combining(c)])
    return texto.upper()

def extraer_sucursal(nombre):
    if not isinstance(nombre, str):
        return "VENDEDOR EXTERNO"
    
    sucursal = None
    
    # 1) Buscar "Mostrador ..."
    match = re.search(r"Mostrador\s+([A-Za-z0-9\s]+)", nombre, re.IGNORECASE)
    if match:
        sucursal = match.group(1).strip()
    else:
        # 2) Buscar "Calle" o "Cota"
        match2 = re.search(r"(Calle\s+\d+|Cota)", nombre, re.IGNORECASE)
        if match2:
            sucursal = match2.group(1).strip()
        else:
            # 3) Buscar prefijo de código
            for prefijo, ciudad in mapa_codigos.items():
                if nombre.upper().startswith(prefijo):
                    return ciudad
            return "VENDEDOR EXTERNO"
    
    # Normalizar texto
    sucursal = normalizar(sucursal)
    
    # Limpiar T1, T2, T3 al final
    sucursal = re.sub(r"\s*T\d+$", "", sucursal).strip()
    
    # Aplicar equivalencias
    sucursal = mapa_equivalencias.get(sucursal, sucursal)
    
    return sucursal

# Aplicar al dataframe
df["Sucursal"] = df["invoice_name"].apply(extraer_sucursal)

In [12]:
df.sample(5)

,price_subtotal,company_id,id,quantity,product_id_num,product_name,invoice_id_num,invoice_name,date_invoice,Sucursal
117507,23900.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1471979,1.0,17658,[DAB04666025] DA4666 FILTRO AIRE 2o. DONSSON B...,400297,FBUC5662,2025-02-14 20:17:35,BUCARAMANGA
192619,236200.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1382978,2.0,18926,[BLS10141125] GS141A FILTROACEITE DUAL VENTUR...,376671,FCOT20941,2024-09-23 17:04:21,VENDEDOR EXTERNO
41970,55000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1561838,2.0,47452,[DLE00726189] G726 FILTRO ACEITE - FREIGHTLINE...,424072,FBAQ13730,2025-07-02 13:31:03,BARRANQUILLA
77097,145800.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1520242,6.0,19512,[BLS00049125] GS049 FILTRO ACEITE JOHN DEERE. ...,413007,FCOT24202,2025-04-28 18:42:20,VENDEDOR EXTERNO
186275,269200.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1390308,2.0,19740,[BHS00133125] GS133 FILTRO HIDRAULICO CARGADOR...,378795,FCAL2406,2024-10-05 14:00:01,CALI


In [13]:
ventas_sucursal = df.groupby("Sucursal")["quantity"].sum()

In [14]:
ventas_sucursal.head(10)

Sucursal
BARRANQUILLA         99503.0
BUCARAMANGA          64832.0
CALI                 46547.0
CALLE 6              58255.0
COTA                 23125.0
MEDELLIN             93158.0
NORTE                23292.0
VALLADOLID           39799.0
VENDEDOR EXTERNO    243785.0
Name: quantity, dtype: float64